In [1]:
from IPython.display import display, Image, clear_output

In [2]:
#Import all the necessary libraries
import pymzml
import csv
import os
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
import re
import plotly.express as px
from collections import defaultdict

import plotly.io as pio
import json
import plotly.graph_objs as go
import matplotlib.colors as mcolors

import json
import ipywidgets as widgets

import warnings

import time
##Custom scripts
# from OzESI_parsing_function_old import *

# from plotting_functions import *

# dfdfplotting_functions


No module named 'ms_deisotope._c.averagine' averagine
No module named 'ms_deisotope._c.scoring'
No module named 'ms_deisotope._c.deconvoluter_base'
No module named 'ms_deisotope._c.deconvoluter_base'
No module named 'ms_deisotope._c.deconvoluter_base'


In [7]:
### Filter for o3on and o2only
# path to Projects/04_29_23/mzml folder

import os
import shutil
print(os.getcwd())
# Set the source directory where the files are located
src_directory = "Projects/04_29_23/mzml/" # For example, '/path/to/your/files'

# Set the destination directories for o3on and o2only
dst_directory_o3on = os.path.join(src_directory, "o3on")
dst_directory_o2only = os.path.join(src_directory, "o2only")

# Create the destination directories if they don't exist
os.makedirs(dst_directory_o3on, exist_ok=True)
os.makedirs(dst_directory_o2only, exist_ok=True)

# Loop through the files in the source directory
for filename in os.listdir(src_directory):
    # Check if the file ends with ".mzML"
    if filename.endswith(".mzML"):
        # Determine which folder to move the file to based on its name
        if "_o3on.mzML" in filename:
            shutil.move(os.path.join(src_directory, filename), os.path.join(dst_directory_o3on, filename))
        elif "_o2only.mzML" in filename:
            shutil.move(os.path.join(src_directory, filename), os.path.join(dst_directory_o2only, filename))





/home/sanjay/github/lipids/Lipidomics/lipid_platform


In [30]:
#OLD WAY
###Importing Variables for all functions

# data_base_name_location = 'lipid_database/Lipid_Database.xlsx'####Lipid database with Standard Carnitines
# mzml_folder = './data_mzml/04-29-2023_mzml/test_oz_1/'
# tolerance = 0.3
# remove_std = True

# # Example usage:
# folder_name_to_save = 'TEST_04-29-2023_validation_test_6'
# file_name_to_save = 'TEST_04-29-2023_validation_test_6'
# save_data= True



# # Example usage:

# file_name_to_save = 'TEST_04-29-2023_validation_test_2'
# save_data= True

data_base_name_location = 'lipid_database/Lipid_Database.xlsx'####Lipid database with Standard Carnitines
#NEW WAY
Pre_folder = './Projects/'
Project_Folder =Pre_folder + '04_29_23/mzml/o3on/'
folder_name_to_save = 'Projects/04_29_23/results/'
file_name_to_save = '04_29_23'


tolerance = 0.3
remove_std = True
save_data= True


In [4]:
# folder_name_to_save = Project_Folder+ 'Processed Results/'
# data_base_name_location = 'lipid_database/Lipid_Database.xlsx'
# mzml_folder = Project_Folder +"mzml/"
# Pre_edge_r_path = Project_Folder+"Pre_EdgeR/"
# plots_2_save_path = Project_Folder+"Plots/"
# label_file = Project_Folder+"Labels/labels.csv"

# ##Labels DF and Labels List
# labels_df = pd.read_csv(label_file)
# labels_list = list(labels_df)
# labels_list = labels_list +["Class","Lipid"]

In [27]:

###All functions

#Function to read in MRM database
#Option to remove STDs from database##Not finished need option to use another database with no qualitative ACs


def read_mrm_list(filename,remove_std = True):
    mrm_list_new = pd.read_excel(filename, sheet_name=None)
    mrm_list_new = pd.concat(mrm_list_new, ignore_index=True)
    mrm_list_offical = mrm_list_new[['Compound Name', 'Parent Ion', 'Product Ion', 'Class']]
    # Add underscore to middle of columns names
    mrm_list_offical.columns = mrm_list_offical.columns.str.replace(' ', '_')
    # Round Parent Ion and Product Ion to 1 decimal place
    mrm_list_offical['Parent_Ion'] = np.round(mrm_list_offical['Parent_Ion'],1)
    mrm_list_offical['Product_Ion'] = np.round(mrm_list_offical['Product_Ion'],1)
    # Create transition column by combining Parent Ion and Product Ion with arrow between numbers
    mrm_list_offical['Transition'] = mrm_list_offical['Parent_Ion'].astype(str) + ' -> ' + mrm_list_offical['Product_Ion'].astype(str)
    # Change column compound name to lipid
    mrm_list_offical = mrm_list_offical.rename(columns={'Compound_Name': 'Lipid'})
    # Make a column called Class match lipid column to lipid types
    if remove_std == True:
        lipid_class = mrm_list_offical['Class'].unique()
        lipid_class_to_keep = ['PS','PG','CE','PC', 'DAG', 'PE', 'TAG', 'FA', 'Cer', 'CAR', 'PI','SM']
        mrm_list_offical = mrm_list_offical[mrm_list_offical['Class'].isin(lipid_class_to_keep)]
    return mrm_list_offical


# Function to create an ion dictionary from an MRM database DataFrame
def create_ion_dict(mrm_database):
    ion_dict = defaultdict(list)
    # Iterate through the rows of the MRM database DataFrame
    for index, row in mrm_database.iterrows():
        # Add a tuple with Lipid and Class to the ion dictionary using Parent_Ion and Product_Ion as the key
        ion_dict[(row['Parent_Ion'], row['Product_Ion'])].append((row['Lipid'], row['Class']))
    return ion_dict

### New way to parse OzESI data
OzESI_time_df = pd.DataFrame(columns=['Lipid', 'Parent_Ion', 'Product_Ion', 'Intensity', 'Transition', 'Class', 'Sample_ID', 'Retention_Time', 'OzESI_Intensity'])

# def mzml_parser(file_name):
#     global OzESI_time_df  # Declare OzESI_time_df as a global variable
    
#     df = pd.DataFrame(columns=['Lipid', 'Parent_Ion', 'Product_Ion', 'Intensity', 'Transition', 'Class', 'Sample_ID'])
#     data_folder = os.listdir(file_name)
#     data_folder.sort()
#     path_to_mzml_files = file_name

#     for file in data_folder:
#         if file.endswith('.mzML'):
#             run = pymzml.run.Reader(path_to_mzml_files + file, skip_chromatogram=False)
#             df_all = pd.DataFrame(columns=['Lipid', 'Parent_Ion', 'Product_Ion', 'Intensity', 'Transition', 'Class', 'Sample_ID'])
#             q1_mz = 0
#             q3_mz = 0
#             count = 0

#             for spectrum in run:
#                 for element in spectrum.ID.split(' '):
#                     intensity_store = np.array([])
                    
#                     if 'Q1' in element:
#                         q1 = element.split('=')
#                         q1_mz = np.round(float(q1[1]), 1)
                    
#                     if 'Q3' in element:
#                         q3 = element.split('=')
#                         q3_mz = np.round(float(q3[1]), 1)
#                         for time, intensity in spectrum.peaks():
#                             row = {
#                                 'Parent_Ion': q1_mz,
#                                 'Product_Ion': q3_mz,
#                                 'Retention_Time': time,
#                                 'OzESI_Intensity': intensity
#                             }
#                             row['Sample_ID'] = file[:-5]
#                             row['Transition'] = str(row['Parent_Ion']) + ' -> ' + str(row['Product_Ion'])
#                             OzESI_time_df = OzESI_time_df.append(row, ignore_index=True)
                        
#                         for mz, intensity in spectrum.peaks():
#                             intensity_store = np.append(intensity_store, intensity)

#                         intensity_sum = np.sum(intensity_store)
#                         df_all.loc[count, 'Parent_Ion'] = q1_mz
#                         df_all.loc[count, 'Product_Ion'] = q3_mz
#                         df_all.loc[count, 'Parent_Ion'] = np.round(df_all.loc[count, 'Parent_Ion'], 1)
#                         df_all.loc[count, 'Product_Ion'] = np.round(df_all.loc[count, 'Product_Ion'], 1)
#                         df_all.loc[count, 'Intensity'] = intensity_sum
#                         df_all.loc[count, 'Transition'] = f"{q1_mz} -> {q3_mz}"
#                         df_all.loc[count, 'Sample_ID'] = file[:-5]
#                         count += 1

#             df = df.append(df_all, ignore_index=True)
#     print('Finished parsing mzML files' + '\n')
#     # print file name to make sure it is working
#     print(file_name)
#     return df

def mzml_parser(file_name):
    global OzESI_time_df  # Declare OzESI_time_df as a global variable
    
    rows = []
    ozesi_rows = []
    
    data_folder = os.listdir(file_name)
    data_folder.sort()
    path_to_mzml_files = file_name

    for file in data_folder:
        if file.endswith('.mzML'):
            run = pymzml.run.Reader(path_to_mzml_files + file, skip_chromatogram=False)
            q1_mz = 0
            q3_mz = 0

            for spectrum in run:
                for element in spectrum.ID.split(' '):
                    
                    if 'Q1' in element:
                        q1 = element.split('=')
                        q1_mz = np.round(float(q1[1]), 1)
                    
                    if 'Q3' in element:
                        q3 = element.split('=')
                        q3_mz = np.round(float(q3[1]), 1)
                        
                        intensity_store = np.array([intensity for _, intensity in spectrum.peaks()])
                        intensity_sum = np.sum(intensity_store)
                        
                        transition = f"{q1_mz} -> {q3_mz}"
                        sample_id = file[:-5]
                        
                        rows.append({
                            'Parent_Ion': q1_mz,
                            'Product_Ion': q3_mz,
                            'Intensity': intensity_sum,
                            'Transition': transition,
                            'Sample_ID': sample_id
                        })
                        
                        for time, intensity in spectrum.peaks():
                            ozesi_rows.append({
                                'Parent_Ion': q1_mz,
                                'Product_Ion': q3_mz,
                                'Retention_Time': time,
                                'OzESI_Intensity': intensity,
                                'Sample_ID': sample_id,
                                'Transition': transition
                            })

    df = pd.DataFrame(rows)
    OzESI_time_df = pd.DataFrame(ozesi_rows)
    print('Finished parsing mzML files\n')
    return df


# Function to check if the absolute difference between two values is within a given tolerance
def within_tolerance(a, b, tolerance=0.3):
    return abs(a - b) <= tolerance

# Function to match the ions in a DataFrame row with the ions in an ion dictionary
def match_ions(row, ion_dict, tolerance=0.3):
    ions = (row['Parent_Ion'], row['Product_Ion'])
    matched_lipids = []
    matched_classes = []

    # Iterate through the ion dictionary
    for key, value in ion_dict.items():
        # Check if both the Parent_Ion and Product_Ion values are within the specified tolerance
        if within_tolerance(ions[0], key[0], tolerance) and within_tolerance(ions[1], key[1], tolerance):
            # If within tolerance, extend the matched_lipids and matched_classes lists with the corresponding values
            matched_lipids.extend([match[0] for match in value])
            matched_classes.extend([match[1] for match in value])

    # If any matches were found, update the Lipid and Class columns in the row
    if matched_lipids and matched_classes:
        row['Lipid'] = ' | '.join(matched_lipids)
        row['Class'] = ' | '.join(matched_classes)

    return row

####Combined functions for Matching

def match_lipids_parser(mrm_database,df, tolerance=0.3):
    ion_dict = create_ion_dict(mrm_database)
    # Assuming you have the df DataFrame to apply the match_ions function
    df_matched = df.apply(lambda row: match_ions(row, ion_dict=ion_dict, tolerance=tolerance), axis=1)


    # df_matched = df_matched.dropna()
    
    return df_matched


def save_dataframe(df, folder_name, file_name, max_attempts=5):
    folder_path = f'data_results/data/data_matching/{folder_name}'
    os.makedirs(folder_path, exist_ok=True)

    for i in range(max_attempts):
        file_path = f'{folder_path}/{file_name}.csv'
        if not os.path.isfile(file_path):
            df.to_csv(file_path, index=False)
            print(f"Saved DataFrame to {file_path}")
            break
    else:
        print(f"Failed to save DataFrame after {max_attempts} attempts.")
        return None


def full_parse(data_base_name_location,mzml_folder, folder_name_to_save, file_name_to_save,tolerance,remove_std = True,
               save_data=False):
    mrm_database = read_mrm_list(data_base_name_location,remove_std=remove_std)
    df = mzml_parser(mzml_folder)
    df_matched = match_lipids_parser(mrm_database,df, tolerance=tolerance)
    
    if save_data == True:
        
        save_dataframe(df_matched, folder_name_to_save, file_name_to_save)

    return df_matched





In [31]:
df_matched = full_parse(data_base_name_location,Project_Folder, folder_name_to_save, 
                        file_name_to_save,tolerance, remove_std = remove_std,save_data=save_data)



/tmp/ipykernel_473/2473317584.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mrm_list_offical['Parent_Ion'] = np.round(mrm_list_offical['Parent_Ion'],1)
/tmp/ipykernel_473/2473317584.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mrm_list_offical['Product_Ion'] = np.round(mrm_list_offical['Product_Ion'],1)
/tmp/ipykernel_473/2473317584.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

Finished parsing mzML files

Failed to save DataFrame after 5 attempts.


In [111]:

df_matched.head(None)
OzESI_time_df.head(None)



,Parent_Ion,Product_Ion,Retention_Time,OzESI_Intensity,Sample_ID,Transition
0,694.6,547.6,0.118533,254.960022,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,694.6 -> 547.6
1,694.6,547.6,0.240367,239.120010,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,694.6 -> 547.6
2,694.6,547.6,0.362183,260.560028,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,694.6 -> 547.6
3,694.6,547.6,0.484017,330.260010,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,694.6 -> 547.6
4,694.6,547.6,0.605833,353.880035,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,694.6 -> 547.6
...,...,...,...,...,...,...
222712,902.8,603.6,19.614183,18169.841797,FAD189_Ms-5xFAD-hippoTG18-1_o3on,902.8 -> 603.6
222713,902.8,603.6,19.736000,18471.162109,FAD189_Ms-5xFAD-hippoTG18-1_o3on,902.8 -> 603.6
222714,902.8,603.6,19.857833,17463.300781,FAD189_Ms-5xFAD-hippoTG18-1_o3on,902.8 -> 603.6
222715,902.8,603.6,19.979650,18173.921875,FAD189_Ms-5xFAD-hippoTG18-1_o3on,902.8 -> 603.6


In [33]:
mrm_database = read_mrm_list(data_base_name_location)
tolerance = 0.3
df_oz_matched = match_lipids_parser(mrm_database, OzESI_time_df, tolerance)


/tmp/ipykernel_473/2473317584.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mrm_list_offical['Parent_Ion'] = np.round(mrm_list_offical['Parent_Ion'],1)
/tmp/ipykernel_473/2473317584.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mrm_list_offical['Product_Ion'] = np.round(mrm_list_offical['Product_Ion'],1)
/tmp/ipykernel_473/2473317584.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

In [113]:
df_oz_matched.tail(50)
# #print to excel
# df_oz_matched.to_excel("OzESI_time_df_04_29_23.xlsx")

,Class,Lipid,OzESI_Intensity,Parent_Ion,Product_Ion,Retention_Time,Sample_ID,Transition
222667,TAG,"[TG(55:10),TG(54:3)]_FA18:1",4010.380371,902.8,603.6,14.131983,FAD189_Ms-5xFAD-hippoTG18-1_o3on,902.8 -> 603.6
222668,TAG,"[TG(55:10),TG(54:3)]_FA18:1",3524.860352,902.8,603.6,14.253817,FAD189_Ms-5xFAD-hippoTG18-1_o3on,902.8 -> 603.6
222669,TAG,"[TG(55:10),TG(54:3)]_FA18:1",4105.120117,902.8,603.6,14.375633,FAD189_Ms-5xFAD-hippoTG18-1_o3on,902.8 -> 603.6
222670,TAG,"[TG(55:10),TG(54:3)]_FA18:1",33123.503906,902.8,603.6,14.497467,FAD189_Ms-5xFAD-hippoTG18-1_o3on,902.8 -> 603.6
222671,TAG,"[TG(55:10),TG(54:3)]_FA18:1",172018.765625,902.8,603.6,14.619283,FAD189_Ms-5xFAD-hippoTG18-1_o3on,902.8 -> 603.6
222672,TAG,"[TG(55:10),TG(54:3)]_FA18:1",175727.187500,902.8,603.6,14.741117,FAD189_Ms-5xFAD-hippoTG18-1_o3on,902.8 -> 603.6
222673,TAG,"[TG(55:10),TG(54:3)]_FA18:1",128808.812500,902.8,603.6,14.862950,FAD189_Ms-5xFAD-hippoTG18-1_o3on,902.8 -> 603.6
222674,TAG,"[TG(55:10),TG(54:3)]_FA18:1",79138.046875,902.8,603.6,14.984767,FAD189_Ms-5xFAD-hippoTG18-1_o3on,902.8 -> 603.6
222675,TAG,"[TG(55:10),TG(54:3)]_FA18:1",55040.562500,902.8,603.6,15.106600,FAD189_Ms-5xFAD-hippoTG18-1_o3on,902.8 -> 603.6
222676,TAG,"[TG(55:10),TG(54:3)]_FA18:1",39817.484375,902.8,603.6,15.228417,FAD189_Ms-5xFAD-hippoTG18-1_o3on,902.8 -> 603.6


In [115]:
#### Filter retention times

filtered_df = df_oz_matched[(df_oz_matched['Retention_Time'] > 10.0) & (df_oz_matched['Retention_Time'] < 16.0)]
filtered_df['Retention_Time'] = filtered_df['Retention_Time'].round(2)
filtered_df['OzESI_Intensity'] = filtered_df['OzESI_Intensity'].round(0)
# Assuming the DataFrame is named df

# Keep only unique combinations of 'Sample_ID' and 'Transition'
#filtered_df = filtered_df.drop_duplicates(subset=['Sample_ID', 'Transition'])


#filtered_df = filtered_df.groupby('Transition').apply(lambda x: x.loc[x['OzESI_Intensity'].idxmax()]).reset_index(drop=True)
filtered_df = filtered_df.groupby(['Sample_ID', 'Transition']).apply(lambda x: x.loc[x['OzESI_Intensity'].idxmax()]).reset_index(drop=True)
filtered_df.tail(100)


/tmp/ipykernel_473/3316649919.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Retention_Time'] = filtered_df['Retention_Time'].round(2)
/tmp/ipykernel_473/3316649919.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['OzESI_Intensity'] = filtered_df['OzESI_Intensity'].round(0)


,Class,Lipid,OzESI_Intensity,Parent_Ion,Product_Ion,Retention_Time,Sample_ID,Transition
1340,NaN,NaN,9851.0,740.8,551.6,14.95,FAD189_M2-5xFAD-diencTG18-1_o3on,740.8 -> 551.6
1341,NaN,NaN,337.0,746.8,599.6,13.12,FAD189_M2-5xFAD-diencTG18-1_o3on,746.8 -> 599.6
1342,NaN,NaN,269.0,748.8,601.6,11.29,FAD189_M2-5xFAD-diencTG18-1_o3on,748.8 -> 601.6
1343,NaN,NaN,698.0,750.8,603.6,10.31,FAD189_M2-5xFAD-diencTG18-1_o3on,750.8 -> 603.6
1344,NaN,NaN,668.0,762.7,573.6,12.63,FAD189_M2-5xFAD-diencTG18-1_o3on,762.7 -> 573.6
...,...,...,...,...,...,...,...,...
1435,TAG,"[TG(53:10),TG(52:3)]_FA18:1",40842.0,874.8,575.6,13.66,FAD189_Ms-5xFAD-hippoTG18-1_o3on,874.8 -> 575.6
1436,TAG,"[TG(53:9),TG(52:2)]_FA18:1",333856.0,876.8,577.6,14.87,FAD189_Ms-5xFAD-hippoTG18-1_o3on,876.8 -> 577.6
1437,TAG,[TG(54:5)]_FA18:1,4512.0,898.8,599.6,12.43,FAD189_Ms-5xFAD-hippoTG18-1_o3on,898.8 -> 599.6
1438,TAG,"[TG(55:11),TG(54:4)]_FA18:1",31386.0,900.8,601.6,13.53,FAD189_Ms-5xFAD-hippoTG18-1_o3on,900.8 -> 601.6


In [116]:
#Create df_matched_2 and concat df_matched and filtered_df
df_matched_2 = pd.concat([df_matched, filtered_df[['Retention_Time', 'OzESI_Intensity']]], axis=1)


In [117]:
df_matched_2.tail(5)

,Class,Intensity,Lipid,Parent_Ion,Product_Ion,Sample_ID,Transition,Retention_Time,OzESI_Intensity
1435,TAG,6.787187e+05,"[TG(53:10),TG(52:3)]_FA18:1",874.8,575.6,FAD189_Ms-5xFAD-hippoTG18-1_o3on,874.8 -> 575.6,13.66,40842.0
1436,TAG,3.647270e+06,"[TG(53:9),TG(52:2)]_FA18:1",876.8,577.6,FAD189_Ms-5xFAD-hippoTG18-1_o3on,876.8 -> 577.6,14.87,333856.0
1437,TAG,1.356127e+05,[TG(54:5)]_FA18:1,898.8,599.6,FAD189_Ms-5xFAD-hippoTG18-1_o3on,898.8 -> 599.6,12.43,4512.0
1438,TAG,3.820187e+05,"[TG(55:11),TG(54:4)]_FA18:1",900.8,601.6,FAD189_Ms-5xFAD-hippoTG18-1_o3on,900.8 -> 601.6,13.53,31386.0
1439,TAG,1.658646e+06,"[TG(55:10),TG(54:3)]_FA18:1",902.8,603.6,FAD189_Ms-5xFAD-hippoTG18-1_o3on,902.8 -> 603.6,14.74,175727.0


In [118]:
# Create a new DataFrame to store the DB_Position and Aldehyde_Ion values
df_DB_position = pd.DataFrame(columns=['DB_Position','Aldehyde_Ion'])

# Loop over the range of DB_Position values and calculate the corresponding Aldehyde_Ion values
for i in range(3, 21):
    df_DB_position.loc[i,'DB_Position'] = i
    df_DB_position.loc[i,'Aldehyde_Ion'] = 26 + (14 * (i-3)) 

# df_DB_position.head(None)

OzESI_list = [7,9,12]

for i in OzESI_list:
        # Retrieve the Aldehyde_Ion value for the current DB_Position
        aldehyde_ion = df_DB_position.loc[df_DB_position["DB_Position"] == i, "Aldehyde_Ion"].values[0]
        
        # Calculate n-i values
        df_matched_2["n-{}".format(i)] = df_matched_2["Parent_Ion"] - aldehyde_ion

df_matched_2.head(None)


,Class,Intensity,Lipid,Parent_Ion,Product_Ion,Sample_ID,Transition,Retention_Time,OzESI_Intensity,n-7,n-9,n-12
0,NaN,2.567996e+05,NaN,694.6,547.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,694.6 -> 547.6,11.69,3903.0,612.6,584.6,542.6
1,NaN,2.185144e+05,NaN,696.6,549.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,696.6 -> 549.6,12.91,18586.0,614.6,586.6,544.6
2,NaN,6.608002e+06,NaN,698.7,551.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,698.7 -> 551.6,14.00,59211.0,616.7,588.7,546.7
3,NaN,7.330655e+04,NaN,720.6,573.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,720.6 -> 573.6,13.63,590.0,638.6,610.6,568.6
4,NaN,1.428391e+05,NaN,722.6,575.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,722.6 -> 575.6,12.77,3643.0,640.6,612.6,570.6
...,...,...,...,...,...,...,...,...,...,...,...,...
1435,TAG,6.787187e+05,"[TG(53:10),TG(52:3)]_FA18:1",874.8,575.6,FAD189_Ms-5xFAD-hippoTG18-1_o3on,874.8 -> 575.6,13.66,40842.0,792.8,764.8,722.8
1436,TAG,3.647270e+06,"[TG(53:9),TG(52:2)]_FA18:1",876.8,577.6,FAD189_Ms-5xFAD-hippoTG18-1_o3on,876.8 -> 577.6,14.87,333856.0,794.8,766.8,724.8
1437,TAG,1.356127e+05,[TG(54:5)]_FA18:1,898.8,599.6,FAD189_Ms-5xFAD-hippoTG18-1_o3on,898.8 -> 599.6,12.43,4512.0,816.8,788.8,746.8
1438,TAG,3.820187e+05,"[TG(55:11),TG(54:4)]_FA18:1",900.8,601.6,FAD189_Ms-5xFAD-hippoTG18-1_o3on,900.8 -> 601.6,13.53,31386.0,818.8,790.8,748.8


In [119]:
#Make a new column for Labels of n-#
df_matched_2['Labels'] = ''
OzESI_list = [7, 9, 12]

def within_tolerance(a, b, tolerance=0.3):
    return abs(a - b) <= tolerance
columns = [
    'Lipid', 'Parent_Ion', 'Product_Ion', 'Intensity', 'Transition', 'Class',
    'Sample_ID', 'Retention_Time', 'Intensity_OzESI', 'Mean_Retention_Time',
    'Mean_Intensity_OzESI', 'n-7', 'n-9', 'n-12', 'Labels'
]
df_OzESI_n = pd.DataFrame(columns=columns)

# def add_lipid_info(df_matched_2, OzESI_list, tolerance=0.3):
#     df_test = df_matched_2.copy()
#     df_test_2 = df_matched_2.copy()
#     global df_OzESI_n
#     for i in OzESI_list:
#         df_test['n-' + str(i)] = df_test['n-' + str(i)].astype(float)
    
#     for i in range(len(df_test)):
#         if pd.isna(df_test.loc[i, 'Lipid']):
#             parent_ion = df_test.loc[i, 'Parent_Ion']
            
#             for j in range(len(df_test)):
#                 row_data = df_test.loc[j].copy()
#                 if within_tolerance(parent_ion, row_data['n-7'], tolerance) and isinstance(row_data['Lipid'], str):
#                     df_test.loc[i, 'Lipid'] = row_data['Lipid']
#                     df_test.loc[i, 'Labels'] = 'n-7' + row_data['Labels']
                    
#                     # Append to df_OzESI_n
#                     appended_row = df_test.loc[i].copy()
#                     appended_row['Labels'] = 'n-7' + row_data['Labels']
#                     df_OzESI_n = df_OzESI_n.append(appended_row, ignore_index=True)
                    
#                 elif within_tolerance(parent_ion, row_data['n-9'], tolerance) and isinstance(row_data['Lipid'], str):
#                     df_test.loc[i, 'Lipid'] = row_data['Lipid']
#                     df_test.loc[i, 'Labels'] = 'n-9' + row_data['Labels']
                    
#                     # Append to df_OzESI_n
#                     appended_row = df_test.loc[i].copy()
#                     appended_row['Labels'] = 'n-9' + row_data['Labels']
#                     df_OzESI_n = df_OzESI_n.append(appended_row, ignore_index=True)
                    
#                 elif within_tolerance(parent_ion, row_data['n-12'], tolerance) and isinstance(row_data['Lipid'], str):
#                     df_test.loc[i, 'Lipid'] = row_data['Lipid']
#                     df_test.loc[i, 'Labels'] = 'n-12' + row_data['Labels']
                    
#                     # Append to df_OzESI_n
#                     appended_row = df_test.loc[i].copy()
#                     appended_row['Labels'] = 'n-12' + row_data['Labels']
#                     df_OzESI_n = df_OzESI_n.append(appended_row, ignore_index=True)
    
#     df_test.dropna(subset=['Lipid'], inplace=True)
#     return df_test

#Function to add lipid name
def add_lipid_info(df_matched_2, OzESI_list, tolerance=0.3):
    df_test = df_matched_2.copy()
    df_test_2 = df_matched_2.copy()
    global df_OzESI_n
    
    for i in OzESI_list:
        df_test['n-' + str(i)] = df_test['n-' + str(i)].astype(float)
    
    for i in range(len(df_test)):
        if pd.isna(df_test.loc[i, 'Lipid']):
            parent_ion = df_test.loc[i, 'Parent_Ion']
            
            for j in range(len(df_test)):
                row_data = df_test.loc[j].copy()
                if within_tolerance(parent_ion, row_data['n-7'], tolerance) and isinstance(row_data['Lipid'], str):
                    df_test.loc[i, 'Lipid'] = row_data['Lipid']
                    df_test.loc[i, 'Labels'] = 'n-7' + row_data['Labels']
                    
                    # Append to df_test_2
                    appended_row = df_test.loc[i].copy()
                    appended_row['Labels'] = 'n-7' + row_data['Labels']
                    df_test_2 = df_test_2.append(appended_row, ignore_index=True)
                    
                elif within_tolerance(parent_ion, row_data['n-9'], tolerance) and isinstance(row_data['Lipid'], str):
                    df_test.loc[i, 'Lipid'] = row_data['Lipid']
                    df_test.loc[i, 'Labels'] = 'n-9' + row_data['Labels']
                    
                    # Append to df_test_2
                    appended_row = df_test.loc[i].copy()
                    appended_row['Labels'] = 'n-9' + row_data['Labels']
                    df_test_2 = df_test_2.append(appended_row, ignore_index=True)
                    
                elif within_tolerance(parent_ion, row_data['n-12'], tolerance) and isinstance(row_data['Lipid'], str):
                    df_test.loc[i, 'Lipid'] = row_data['Lipid']
                    df_test.loc[i, 'Labels'] = 'n-12' + row_data['Labels']
                    
                    # Append to df_test_2
                    appended_row = df_test.loc[i].copy()
                    appended_row['Labels'] = 'n-12' + row_data['Labels']
                    df_test_2 = df_test_2.append(appended_row, ignore_index=True)
    
    df_test_2.dropna(subset=['Lipid'], inplace=True)
    return df_test_2


In [120]:
#Add lipid name to df_matched 3
df_matched_3 = add_lipid_info(df_matched_2, OzESI_list, tolerance=0.3)

/tmp/ipykernel_473/2444659214.py:97: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_test_2 = df_test_2.append(appended_row, ignore_index=True)
/tmp/ipykernel_473/2444659214.py:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_test_2 = df_test_2.append(appended_row, ignore_index=True)
/tmp/ipykernel_473/2444659214.py:79: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_test_2 = df_test_2.append(appended_row, ignore_index=True)


In [122]:
df_matched_3.head(None) 

,Class,Intensity,Lipid,Parent_Ion,Product_Ion,Sample_ID,Transition,Retention_Time,OzESI_Intensity,n-7,n-9,n-12,Labels
27,TAG,6.869138e+05,[TG(50:3)]_FA18:1,846.8,547.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,846.8 -> 547.6,12.58,72587.0,764.8,736.8,694.8,
28,TAG,3.262575e+06,"[TG(51:9),TG(50:2)]_FA18:1",848.8,549.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,848.8 -> 549.6,13.79,409651.0,766.8,738.8,696.8,
29,TAG,8.906404e+06,"[TG(51:8),TG(50:1)]_FA18:1",850.8,551.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,850.8 -> 551.6,15.13,2236988.0,768.8,740.8,698.8,
30,TAG,7.219475e+05,[TG(52:4)]_FA18:1,872.8,573.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,872.8 -> 573.6,12.57,159733.0,790.8,762.8,720.8,
31,TAG,4.364672e+06,"[TG(53:10),TG(52:3)]_FA18:1",874.8,575.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,874.8 -> 575.6,13.66,928619.0,792.8,764.8,722.8,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58995,NaN,7.496407e+04,"[TG(55:10),TG(54:3)]_FA18:1",820.8,603.6,FAD189_Ms-5xFAD-hippoTG18-1_o3on,820.8 -> 603.6,14.65,1985.0,738.8,710.8,668.8,n-7
58996,NaN,7.496407e+04,"[TG(55:10),TG(54:3)]_FA18:1",820.8,603.6,FAD189_Ms-5xFAD-hippoTG18-1_o3on,820.8 -> 603.6,14.65,1985.0,738.8,710.8,668.8,n-7
58997,NaN,7.496407e+04,"[TG(55:10),TG(54:3)]_FA18:1",820.8,603.6,FAD189_Ms-5xFAD-hippoTG18-1_o3on,820.8 -> 603.6,14.65,1985.0,738.8,710.8,668.8,n-7
58998,NaN,7.496407e+04,"[TG(55:10),TG(54:3)]_FA18:1",820.8,603.6,FAD189_Ms-5xFAD-hippoTG18-1_o3on,820.8 -> 603.6,14.65,1985.0,738.8,710.8,668.8,n-7


In [127]:
df_matched_3_sorted = df_matched_3.sort_values(by=['Sample_ID','Product_Ion'])
#drop duplicates Lipid and Labels
df_matched_3_sorted = df_matched_3_sorted.drop_duplicates()
#df_matched_3_sorted = df_matched_3_sorted.drop_duplicates(subset=['Lipid','Labels'], keep='first')
df_matched_3_sorted.head(None)


,Class,Intensity,Lipid,Parent_Ion,Product_Ion,Sample_ID,Transition,Retention_Time,OzESI_Intensity,n-7,n-9,n-12,Labels
27,TAG,6.869138e+05,[TG(50:3)]_FA18:1,846.8,547.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,846.8 -> 547.6,12.58,72587.0,764.8,736.8,694.8,
1440,NaN,2.567996e+05,[TG(50:3)]_FA18:1,694.6,547.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,694.6 -> 547.6,11.69,3903.0,612.6,584.6,542.6,n-12
1680,NaN,6.613184e+05,[TG(50:3)]_FA18:1,736.7,547.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,736.7 -> 547.6,12.52,9713.0,654.7,626.7,584.7,n-9
1960,NaN,1.187571e+06,[TG(50:3)]_FA18:1,764.7,547.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,764.7 -> 547.6,15.79,9290.0,682.7,654.7,612.7,n-7
1961,NaN,1.187571e+06,"[TG(53:10),TG(52:3)]_FA18:1",764.7,547.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,764.7 -> 547.6,15.79,9290.0,682.7,654.7,612.7,n-9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1439,TAG,1.658646e+06,"[TG(55:10),TG(54:3)]_FA18:1",902.8,603.6,FAD189_Ms-5xFAD-hippoTG18-1_o3on,902.8 -> 603.6,14.74,175727.0,820.8,792.8,750.8,
58040,NaN,6.577386e+04,"[TG(55:10),TG(54:3)]_FA18:1",750.8,603.6,FAD189_Ms-5xFAD-hippoTG18-1_o3on,750.8 -> 603.6,10.31,697.0,668.8,640.8,598.8,n-12
58760,NaN,1.684532e+05,"[TG(53:10),TG(52:3)]_FA18:1",792.8,603.6,FAD189_Ms-5xFAD-hippoTG18-1_o3on,792.8 -> 603.6,14.66,17906.0,710.8,682.8,640.8,n-7
58761,NaN,1.684532e+05,"[TG(55:10),TG(54:3)]_FA18:1",792.8,603.6,FAD189_Ms-5xFAD-hippoTG18-1_o3on,792.8 -> 603.6,14.66,17906.0,710.8,682.8,640.8,n-9


In [128]:
df_matched_3_sorted.head(None)

,Class,Intensity,Lipid,Parent_Ion,Product_Ion,Sample_ID,Transition,Retention_Time,OzESI_Intensity,n-7,n-9,n-12,Labels
27,TAG,6.869138e+05,[TG(50:3)]_FA18:1,846.8,547.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,846.8 -> 547.6,12.58,72587.0,764.8,736.8,694.8,
1440,NaN,2.567996e+05,[TG(50:3)]_FA18:1,694.6,547.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,694.6 -> 547.6,11.69,3903.0,612.6,584.6,542.6,n-12
1680,NaN,6.613184e+05,[TG(50:3)]_FA18:1,736.7,547.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,736.7 -> 547.6,12.52,9713.0,654.7,626.7,584.7,n-9
1960,NaN,1.187571e+06,[TG(50:3)]_FA18:1,764.7,547.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,764.7 -> 547.6,15.79,9290.0,682.7,654.7,612.7,n-7
1961,NaN,1.187571e+06,"[TG(53:10),TG(52:3)]_FA18:1",764.7,547.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,764.7 -> 547.6,15.79,9290.0,682.7,654.7,612.7,n-9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1439,TAG,1.658646e+06,"[TG(55:10),TG(54:3)]_FA18:1",902.8,603.6,FAD189_Ms-5xFAD-hippoTG18-1_o3on,902.8 -> 603.6,14.74,175727.0,820.8,792.8,750.8,
58040,NaN,6.577386e+04,"[TG(55:10),TG(54:3)]_FA18:1",750.8,603.6,FAD189_Ms-5xFAD-hippoTG18-1_o3on,750.8 -> 603.6,10.31,697.0,668.8,640.8,598.8,n-12
58760,NaN,1.684532e+05,"[TG(53:10),TG(52:3)]_FA18:1",792.8,603.6,FAD189_Ms-5xFAD-hippoTG18-1_o3on,792.8 -> 603.6,14.66,17906.0,710.8,682.8,640.8,n-7
58761,NaN,1.684532e+05,"[TG(55:10),TG(54:3)]_FA18:1",792.8,603.6,FAD189_Ms-5xFAD-hippoTG18-1_o3on,792.8 -> 603.6,14.66,17906.0,710.8,682.8,640.8,n-9


In [129]:
#def compute_intensity_ratio(df):

# df_matched_4 = df_matched_3.copy()  # make a copy of the dataframe to avoid changing the original one
# df['Ratio'] = None  # initially set all ratios to None
# unique_lipids = df['Lipid'].unique()
# for lipid in unique_lipids:
#     lipid_rows = df[df['Lipid'] == lipid]
#     if 'n-7' in lipid_rows['Labels'].values and 'n-9' in lipid_rows['Labels'].values:
#         # check if the 'Sample_ID' values are the same for 'n-7' and 'n-9' lipid rows
#         sample_id_n7 = lipid_rows[lipid_rows['Labels'] == 'n-7']['Sample_ID'].values[0]
#         sample_id_n9 = lipid_rows[lipid_rows['Labels'] == 'n-9']['Sample_ID'].values[0]
#         if sample_id_n7 == sample_id_n9:
#             intensity_n7 = lipid_rows[lipid_rows['Labels'] == 'n-7']['Intensity_OzESI'].values[0]
#             intensity_n9 = lipid_rows[lipid_rows['Labels'] == 'n-9']['Intensity_OzESI'].values[0]
            
#             # Extract scalar values from lists if they are in list format
#             if isinstance(intensity_n7, list) and len(intensity_n7) > 0:
#                 intensity_n7 = intensity_n7[0]
#             if isinstance(intensity_n9, list) and len(intensity_n9) > 0:
#                 intensity_n9 = intensity_n9[0]

#             # Check if the intensities are available and non-zero before dividing
#             if intensity_n7 and intensity_n9 and intensity_n7 != 0:
#                 intensity_ratio = round(intensity_n9 / intensity_n7, 2)  # rounding to 2 decimal places
#                 df.loc[lipid_rows[lipid_rows['Labels'] == 'n-7'].index, 'Ratio'] = intensity_ratio
#                 print(f'Lipid: {lipid}, Intensity Ratio (n-9/n-7): {intensity_ratio}')


df_matched_4 = df_matched_3_sorted.copy()
df_matched_4['Ratios'] = None
# unique_lipids = df_matched_4['Lipid'].unique()

# for lipid in unique_lipids:
#     lipid_rows = df_matched_4[df_matched_4['Lipid'] == lipid]

#     if 'n-7' in lipid_rows['Labels'].values and 'n-9' in lipid_rows['Labels'].values:
#         # Get all indices of 'n-7' and 'n-9' rows within the lipid group
#         n7_indices = lipid_rows[lipid_rows['Labels'] == 'n-7'].index
#         n9_indices = lipid_rows[lipid_rows['Labels'] == 'n-9'].index

#         for n7_idx in n7_indices:
#             for n9_idx in n9_indices:
#                 if lipid_rows.loc[n7_idx, 'Sample_ID'] == lipid_rows.loc[n9_idx, 'Sample_ID']:
#                     intensity_n7 = lipid_rows.loc[n7_idx, 'OzESI_Intensity']
#                     intensity_n9 = lipid_rows.loc[n9_idx, 'OzESI_Intensity']

#                     if isinstance(intensity_n7, list) and len(intensity_n7) > 0:
#                         intensity_n7 = intensity_n7[0]

#                     if isinstance(intensity_n9, list) and len(intensity_n9) > 0:
#                         intensity_n9 = intensity_n9[0]

#                     if intensity_n7 and intensity_n9 and intensity_n7 != 0:
#                         intensity_ratio = round(intensity_n9 / intensity_n7, 2)
#                         df_matched_4.loc[n7_idx, 'Ratio'] = intensity_ratio
#                         print(f"Lipid: {lipid}, Intensity Ratio (n-9/n-7): {intensity_ratio}")





def calculate_intensity_ratio(df):
    # Create a new column for ratios
    df['Ratios'] = pd.Series(dtype='float64')

    # Iterate through each row in the DataFrame
    for index, row in df.iterrows():
        lipid = row['Lipid']
        label = row['Labels']
        intensity = row['OzESI_Intensity']

        # Check if the label is n-9
        if label == 'n-9':
            # Find the corresponding row with n-7 label and same lipid name
            n7_row = df[(df['Lipid'] == lipid) & (df['Labels'] == 'n-7')]

            # If a matching row is found, calculate the intensity ratio
            if not n7_row.empty:
                n7_intensity = n7_row['OzESI_Intensity'].values[0]
                ratio = intensity / n7_intensity

                # Assign the ratio to the 'Ratios' column
                df.at[index, 'Ratios'] = ratio

    return df

df_matched_4 = calculate_intensity_ratio(df_matched_4)


In [131]:
df_matched_4.head(None)

,Class,Intensity,Lipid,Parent_Ion,Product_Ion,Sample_ID,Transition,Retention_Time,OzESI_Intensity,n-7,n-9,n-12,Labels,Ratios
27,TAG,6.869138e+05,[TG(50:3)]_FA18:1,846.8,547.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,846.8 -> 547.6,12.58,72587.0,764.8,736.8,694.8,,NaN
1440,NaN,2.567996e+05,[TG(50:3)]_FA18:1,694.6,547.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,694.6 -> 547.6,11.69,3903.0,612.6,584.6,542.6,n-12,NaN
1680,NaN,6.613184e+05,[TG(50:3)]_FA18:1,736.7,547.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,736.7 -> 547.6,12.52,9713.0,654.7,626.7,584.7,n-9,1.045533
1960,NaN,1.187571e+06,[TG(50:3)]_FA18:1,764.7,547.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,764.7 -> 547.6,15.79,9290.0,682.7,654.7,612.7,n-7,NaN
1961,NaN,1.187571e+06,"[TG(53:10),TG(52:3)]_FA18:1",764.7,547.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,764.7 -> 547.6,15.79,9290.0,682.7,654.7,612.7,n-9,0.494623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1439,TAG,1.658646e+06,"[TG(55:10),TG(54:3)]_FA18:1",902.8,603.6,FAD189_Ms-5xFAD-hippoTG18-1_o3on,902.8 -> 603.6,14.74,175727.0,820.8,792.8,750.8,,NaN
58040,NaN,6.577386e+04,"[TG(55:10),TG(54:3)]_FA18:1",750.8,603.6,FAD189_Ms-5xFAD-hippoTG18-1_o3on,750.8 -> 603.6,10.31,697.0,668.8,640.8,598.8,n-12,NaN
58760,NaN,1.684532e+05,"[TG(53:10),TG(52:3)]_FA18:1",792.8,603.6,FAD189_Ms-5xFAD-hippoTG18-1_o3on,792.8 -> 603.6,14.66,17906.0,710.8,682.8,640.8,n-7,NaN
58761,NaN,1.684532e+05,"[TG(55:10),TG(54:3)]_FA18:1",792.8,603.6,FAD189_Ms-5xFAD-hippoTG18-1_o3on,792.8 -> 603.6,14.66,17906.0,710.8,682.8,640.8,n-9,0.690871


In [136]:
# Filter rows where 'Ratios' column is not NaN
df_matched_5 = df_matched_4[df_matched_4['Ratios'].notna()]

df_matched_5.to_excel("df_matched_5.xlsx")

In [145]:
df_matched_5.head(None)

,Class,Intensity,Lipid,Parent_Ion,Product_Ion,Sample_ID,Transition,Retention_Time,OzESI_Intensity,n-7,n-9,n-12,Labels,Ratios
1680,NaN,6.613184e+05,[TG(50:3)]_FA18:1,736.7,547.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,736.7 -> 547.6,12.52,9713.0,654.7,626.7,584.7,n-9,1.045533
1961,NaN,1.187571e+06,"[TG(53:10),TG(52:3)]_FA18:1",764.7,547.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,764.7 -> 547.6,15.79,9290.0,682.7,654.7,612.7,n-9,0.494623
1720,NaN,4.582103e+05,"[TG(51:9),TG(50:2)]_FA18:1",738.7,549.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,738.7 -> 549.6,13.74,49935.0,656.7,628.7,586.7,n-9,3.244007
2121,NaN,3.307983e+05,"[TG(53:9),TG(52:2)]_FA18:1",766.7,549.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,766.7 -> 549.6,13.71,15393.0,684.7,656.7,614.7,n-9,0.155710
1760,NaN,9.366814e+05,"[TG(51:8),TG(50:1)]_FA18:1",740.8,551.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,740.8 -> 551.6,14.95,144069.0,658.8,630.8,588.8,n-9,1.729541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58681,NaN,6.535404e+04,"[TG(55:10),TG(54:3)]_FA18:1",792.7,575.6,FAD189_Ms-5xFAD-hippoTG18-1_o3on,792.7 -> 575.6,13.57,815.0,710.7,682.7,640.7,n-9,0.031445
58361,NaN,3.525918e+05,"[TG(53:9),TG(52:2)]_FA18:1",766.7,577.6,FAD189_Ms-5xFAD-hippoTG18-1_o3on,766.7 -> 577.6,14.81,25329.0,684.7,656.7,614.7,n-9,0.256219
58480,NaN,5.632846e+04,[TG(54:5)]_FA18:1,788.8,599.6,FAD189_Ms-5xFAD-hippoTG18-1_o3on,788.8 -> 599.6,12.48,576.0,706.8,678.8,636.8,n-9,0.194463
58601,NaN,6.710156e+04,"[TG(55:11),TG(54:4)]_FA18:1",790.8,601.6,FAD189_Ms-5xFAD-hippoTG18-1_o3on,790.8 -> 601.6,13.57,1878.0,708.8,680.8,638.8,n-9,0.532917


In [153]:
df_matched_6 = df_matched_5.copy()
# Assuming df_matched_5 is your DataFrame

df_matched_6 = df_matched_6.sort_values(by='Ratios', ascending=False)
# Remove duplicates based on Sample_ID and Lipid
df_matched_6 = df_matched_6.drop_duplicates(subset=['Sample_ID', 'Lipid'])
df_matched_6 = df_matched_6.sort_values(by='Sample_ID', ascending=True)

df_matched_6.head(None)


,Class,Intensity,Lipid,Parent_Ion,Product_Ion,Sample_ID,Transition,Retention_Time,OzESI_Intensity,n-7,n-9,n-12,Labels,Ratios
2441,NaN,195273.793320,"[TG(55:11),TG(54:4)]_FA18:1",790.8,601.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,790.8 -> 601.6,12.48,16153.0,708.8,680.8,638.8,n-9,4.583712
2601,NaN,748905.374359,"[TG(55:10),TG(54:3)]_FA18:1",792.8,603.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,792.8 -> 603.6,14.66,116813.0,710.8,682.8,640.8,n-9,4.507022
1720,NaN,458210.334137,"[TG(51:9),TG(50:2)]_FA18:1",738.7,549.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,738.7 -> 549.6,13.74,49935.0,656.7,628.7,586.7,n-9,3.244007
1920,NaN,117382.528793,[TG(52:4)]_FA18:1,762.7,573.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,762.7 -> 573.6,12.50,4476.0,680.7,652.7,610.7,n-9,3.053206
2320,NaN,148250.589539,[TG(54:5)]_FA18:1,788.8,599.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,788.8 -> 599.6,12.36,8898.0,706.8,678.8,636.8,n-9,3.004051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58080,NaN,69821.684830,[TG(52:4)]_FA18:1,762.7,573.6,FAD189_Ms-5xFAD-hippoTG18-1_o3on,762.7 -> 573.6,15.67,555.0,680.7,652.7,610.7,n-9,0.378581
58601,NaN,67101.564667,"[TG(55:11),TG(54:4)]_FA18:1",790.8,601.6,FAD189_Ms-5xFAD-hippoTG18-1_o3on,790.8 -> 601.6,13.57,1878.0,708.8,680.8,638.8,n-9,0.532917
58121,NaN,938015.985718,"[TG(53:10),TG(52:3)]_FA18:1",764.7,547.6,FAD189_Ms-5xFAD-hippoTG18-1_o3on,764.7 -> 547.6,15.79,8868.0,682.7,654.7,612.7,n-9,0.472154
57920,NaN,311655.842590,"[TG(51:8),TG(50:1)]_FA18:1",740.8,551.6,FAD189_Ms-5xFAD-hippoTG18-1_o3on,740.8 -> 551.6,14.95,9913.0,658.8,630.8,588.8,n-9,0.119005


In [183]:
df_matched_7 = df_matched_6.copy()
df_matched_7 = df_matched_7.sort_values(by=['Sample_ID', 'Lipid'], ascending=[True, True])

def sort_by_second_tg(lipid):
    tgs = lipid.split(',')
    if len(tgs) > 1:
        return tgs[1]
    else:
        return lipid

df_matched_7['Lipid'] = df_matched_7['Lipid'].apply(sort_by_second_tg)
df_matched_7 = df_matched_7.sort_values(by=['Sample_ID', 'Lipid'], ascending=[True, True])
df_matched_7.head(None)


,Class,Intensity,Lipid,Parent_Ion,Product_Ion,Sample_ID,Transition,Retention_Time,OzESI_Intensity,n-7,n-9,n-12,Labels,Ratios
1760,NaN,9.366814e+05,TG(50:1)]_FA18:1,740.8,551.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,740.8 -> 551.6,14.95,144069.0,658.8,630.8,588.8,n-9,1.729541
1720,NaN,4.582103e+05,TG(50:2)]_FA18:1,738.7,549.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,738.7 -> 549.6,13.74,49935.0,656.7,628.7,586.7,n-9,3.244007
2201,NaN,1.179938e+06,TG(52:2)]_FA18:1,766.7,577.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,766.7 -> 577.6,14.81,266662.0,684.7,656.7,614.7,n-9,2.697452
2041,NaN,2.812500e+05,TG(52:3)]_FA18:1,764.7,575.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,764.7 -> 575.6,13.59,26821.0,682.7,654.7,612.7,n-9,1.428016
2601,NaN,7.489054e+05,TG(54:3)]_FA18:1,792.8,603.6,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,792.8 -> 603.6,14.66,116813.0,710.8,682.8,640.8,n-9,4.507022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58761,NaN,1.684532e+05,TG(54:3)]_FA18:1,792.8,603.6,FAD189_Ms-5xFAD-hippoTG18-1_o3on,792.8 -> 603.6,14.66,17906.0,710.8,682.8,640.8,n-9,0.690871
58601,NaN,6.710156e+04,TG(54:4)]_FA18:1,790.8,601.6,FAD189_Ms-5xFAD-hippoTG18-1_o3on,790.8 -> 601.6,13.57,1878.0,708.8,680.8,638.8,n-9,0.532917
57763,NaN,4.825633e+05,[TG(50:3)]_FA18:1,736.7,547.6,FAD189_Ms-5xFAD-hippoTG18-1_o3on,736.7 -> 547.6,15.94,4474.0,654.7,626.7,584.7,n-9,0.481593
58080,NaN,6.982168e+04,[TG(52:4)]_FA18:1,762.7,573.6,FAD189_Ms-5xFAD-hippoTG18-1_o3on,762.7 -> 573.6,15.67,555.0,680.7,652.7,610.7,n-9,0.378581


In [187]:
df_matched_7.to_excel("df_matched_7.xlsx")

In [184]:
import pandas as pd
import numpy as np

# Assume your DataFrame is named df

# Iterate through each row in the DataFrame
for index, row in df_matched_7.iterrows():
    # Extract Lipid, Sample_ID, Labels and Ratios from the row
    lipid = row['Lipid']
    sample_id = row['Sample_ID']
    labels = row['Labels']
    ratios = row['Ratios']

    # Check if ratios is not NaN
    if not pd.isna(ratios):
        # Print out the values
        print(f'Lipid: {lipid}, Sample_ID: {sample_id}, Labels: {labels}, Ratios: {ratios}')
    else:
        # Print out the values without ratios
        print(f'Lipid: {lipid}, Sample_ID: {sample_id}, Labels: {labels}')


Lipid: TG(50:1)]_FA18:1, Sample_ID: DOD93_F4-5xFAD-Cereb_TG18-1_o3on, Labels: n-9, Ratios: 1.7295405707151346
Lipid: TG(50:2)]_FA18:1, Sample_ID: DOD93_F4-5xFAD-Cereb_TG18-1_o3on, Labels: n-9, Ratios: 3.244007016176184
Lipid: TG(52:2)]_FA18:1, Sample_ID: DOD93_F4-5xFAD-Cereb_TG18-1_o3on, Labels: n-9, Ratios: 2.6974518749304552
Lipid: TG(52:3)]_FA18:1, Sample_ID: DOD93_F4-5xFAD-Cereb_TG18-1_o3on, Labels: n-9, Ratios: 1.428016185709722
Lipid: TG(54:3)]_FA18:1, Sample_ID: DOD93_F4-5xFAD-Cereb_TG18-1_o3on, Labels: n-9, Ratios: 4.507022146770584
Lipid: TG(54:4)]_FA18:1, Sample_ID: DOD93_F4-5xFAD-Cereb_TG18-1_o3on, Labels: n-9, Ratios: 4.583711691259932
Lipid: [TG(50:3)]_FA18:1, Sample_ID: DOD93_F4-5xFAD-Cereb_TG18-1_o3on, Labels: n-9, Ratios: 1.0455328310010765
Lipid: [TG(52:4)]_FA18:1, Sample_ID: DOD93_F4-5xFAD-Cereb_TG18-1_o3on, Labels: n-9, Ratios: 3.053206002728513
Lipid: [TG(54:5)]_FA18:1, Sample_ID: DOD93_F4-5xFAD-Cereb_TG18-1_o3on, Labels: n-9, Ratios: 3.0040513166779204
Lipid: TG(50

Visualizations

In [186]:
import plotly.express as px
import os

# Create the "Plots" directory if it doesn't exist
os.makedirs("Projects/04_29_23/Plots/filtered_lipids/", exist_ok=True)


# Define color mappings for Lipid patterns
color_mapping = {
    '50': 'red',
    '51': 'brown',
    '52': 'blue',
    '53': 'purple',
    '54': 'green',
}

# Get the unique Sample_IDs
sample_ids = df_matched_7['Sample_ID'].unique()

# Loop over the unique Sample_IDs
for sample_id in sample_ids:
    
    # Filter the dataframe for the current Sample_ID
    df_sample = df_matched_7[df_matched_7['Sample_ID'] == sample_id]
    
    # Assign colors to Lipids based on patterns
    lipid_colors = []
    for lipid in df_sample['Lipid']:
        color = 'gray'  # Default color
        for pattern, pattern_color in color_mapping.items():
            if pattern in lipid:
                color = pattern_color
                break
        lipid_colors.append(color)
    
    # Create the bar plot
    fig = px.bar(df_sample, x='Lipid', y='Ratios', text='Ratios', title=f'Bar Plot for Sample_ID: {sample_id}',
                 color_discrete_map=color_mapping)
    
    # Apply colors to the bars
    fig.update_traces(
        marker_color=lipid_colors,
        texttemplate='%{text:.2f}',
        textposition='auto',
        marker_line_width=0
    )
    
    # Customize the layout
    fig.update_layout(
        uniformtext_minsize=18,
        uniformtext_mode='hide',
        xaxis=dict(
            title='Lipid',
            titlefont=dict(size=16)
        ),
        yaxis=dict(
            title='Ratios',
            titlefont=dict(size=16),
            tickfont=dict(size=16)  # Set the font size of y-axis labels
        ),
        legend=dict(
            title='Lipid Patterns',
            tracegroupgap=50,
            itemsizing='constant'
        ),
        title=dict(
            text=f'Sample_ID: {sample_id}',
            font=dict(size=20)  # Set the title font size
        )
    )
    # Save the plot as an image
    file_name = f"Projects/04_29_23/Plots/filtered_lipids//plot_{sample_id}.png"
    fig.write_image(file_name)
    
    # Show plot
    fig.show()


In [ ]:
#########################

In [ ]:
##########################
##########################
##########################
##########################
##########################
##########################

In [8]:
###Duplicate code?

# def process_chromatogram(OzESI_time):
#     # Create dataframe from OzESI_time dictionary
#     OzESI_rt_df = pd.DataFrame(list(OzESI_time.items()), columns=['Retention_Time', 'intensity'])
    
#     # Split intensity column into three columns intensity, Parent_Ion and Product_Ion
#     OzESI_rt_df[['intensity','Parent_Ion','Product_Ion']] = pd.DataFrame(OzESI_rt_df['intensity'].tolist(), index=OzESI_rt_df.index)
    
#     # Round retention Retention_Time to 1 decimal place
#     OzESI_rt_df['Retention_Time'] = round(OzESI_rt_df['Retention_Time'], 2)
    
#     # Create a column called Transition with the Parent_Ion and Product_Ion
#     OzESI_rt_df['Transition'] = OzESI_rt_df['Parent_Ion'].astype(str) + ' -> ' + OzESI_rt_df['Product_Ion'].astype(str)
    
#     ########### HARDCODED TO DROP RETENTION TIMES BELOW 7 SECONDS ############
#     #drop Rention_Time below 10.5 seconds and above 15.5 seconds
#     OzESI_rt_df = OzESI_rt_df[OzESI_rt_df['Retention_Time'] > 10.5]
#     OzESI_rt_df = OzESI_rt_df[OzESI_rt_df['Retention_Time'] < 15.5]

#     # Get the top 10 records for each 'Transition' based on 'intensity'
#     OzESI_rt_df_top = OzESI_rt_df.groupby('Transition').apply(lambda x: x.nlargest(1, 'intensity')).reset_index(drop=True)

#     print('OzESI 1 largest per transition: \n', OzESI_rt_df_top)

#     # peaks, _ = find_peaks(OzESI_rt_df['intensity'], height=0.5e5,distance=1000)
#     # plt.scatter(OzESI_rt_df['Retention_Time'] ,OzESI_rt_df['intensity'])
#     # plt.scatter(OzESI_rt_df.iloc[peaks]['Retention_Time'], OzESI_rt_df.iloc[peaks]['intensity'], "x")
#     # plt.ylabel('Intensity')
#     # plt.xlabel('Retention Time')
#     # plt.title('OzESI LC Chromatogram')
#     # plt.show()
    
#     return OzESI_rt_df_top


# def add_rt_intensity(df, OzESI_rt_df_top):

#     #### STRING OF RETENTION TIMES
#     # Group by 'Transition' and get lists of retention times and intensities
#     transitions_to_rt = OzESI_rt_df_top.groupby('Transition')['Retention_Time'].apply(list).to_dict()
#     transitions_to_intensity = OzESI_rt_df_top.groupby('Transition')['intensity'].apply(list).to_dict()
    
#     # Use the map function to add retention times and intensities to the dataframe as strings
#     df['Retention_Time'] = df['Transition'].map(transitions_to_rt).apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
#     df['Intensity_OzESI'] = df['Transition'].map(transitions_to_intensity).apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
    

#     #MEAN OF RENETION TIMES
#     # Convert the 'Retention_Time' column from string of lists to lists of floats
#     df['Retention_Time'] = df['Retention_Time'].apply(lambda x: [float(i) for i in x.split(', ')] if isinstance(x, str) else x)

#     # Calculate the mean of 'Retention_Time' for each transition
#     df['Mean_Retention_Time'] = df['Retention_Time'].apply(np.mean).round(2)

#     # Convert the 'Intensity_OzESI' column from string of lists to lists of floats
#     df['Intensity_OzESI'] = df['Intensity_OzESI'].apply(lambda x: [float(i) for i in x.split(', ')] if isinstance(x, str) else x)

#     # Calculate the mean of 'Intensity_OzESI' for each transition
#     df['Mean_Intensity_OzESI'] = df['Intensity_OzESI'].apply(np.mean)

#     return df

# def create_aldehyde_ion_dataframe():
#     # Create a pandas dataframe with columns for DB_Position and Aldehyde_Ion
#     df_OzESI = pd.DataFrame(columns=['DB_Position','Aldehyde_Ion'])

#     # Loop over the range of DB_Position values and calculate the corresponding Aldehyde_Ion values
#     for i in range(3, 21):
#         df_OzESI.loc[i,'DB_Position'] = i
#         df_OzESI.loc[i,'Aldehyde_Ion'] = 26 + (14 * (i-3)) 

#     # Print the first 25 rows of the dataframe
#     # print(df_OzESI.head(25))

#     # Return the dataframe
#     return df_OzESI

# #OzESI_list = [3,5,7,9,11]
# OzESI_list = [7,9,12]
# #def calculate_n_minus_values(df_matched, df_OzESI, OzESI_list=[3,5,7,9,11], starting_column=9, last_column=14):
# def calculate_n_minus_values(df_matched, df_OzESI, OzESI_list=[7,9,12], starting_column=9, last_column=14):
#     """
#     Given a pandas dataframe df and a dataframe df_OzESI containing DB_Position and Aldehyde_Ion values,
#     calculates the n-i values for each i in OzESI_list by subtracting the corresponding Aldehyde_Ion value
#     from the Parent_Ion column in df and storing the result in new columns named 'n-i' in df.
#     The starting_column and last_column parameters specify the range of columns in which the n-i values should be stored.
#     """
#     # Create new columns in df for n-i values
#     for i in OzESI_list:
#         df_matched[f"n-{i}"] = df_matched["Parent_Ion"] - df_OzESI.loc[df_OzESI["DB_Position"] == i, "Aldehyde_Ion"].values[0]
    
#     return df_matched



In [37]:

# ##first try
# def process_chromatogram(OzESI_time_df):
#     # Create a copy of the DataFrame to avoid modifying the original DataFrame
#     OzESI_rt_df = OzESI_time_df.copy()

#     # Round retention Retention_Time to 1 decimal place
#     OzESI_rt_df['Retention_Time'] = round(OzESI_rt_df['Retention_Time'], 2)

#     # Create a column called Transition with the Parent_Ion and Product_Ion
#     OzESI_rt_df['Transition'] = OzESI_rt_df['Parent_Ion'].astype(str) + ' -> ' + OzESI_rt_df['Product_Ion'].astype(str)

#     # Drop Retention_Time values below 10.0 seconds and above 15.5 seconds
#     OzESI_rt_df = OzESI_rt_df[(OzESI_rt_df['Retention_Time'] > 10.0) & (OzESI_rt_df['Retention_Time'] < 15.5)]

#     # Get the top 1 record for each 'Transition' based on 'OzESI_Intensity'
#     OzESI_rt_df_top = OzESI_rt_df.groupby('Transition').apply(lambda x: x.nlargest(1, 'OzESI_Intensity')).reset_index(drop=True)

#     print('OzESI 1 largest per transition:\n', OzESI_rt_df_top)

#     return OzESI_rt_df_top

###second try
def process_chromatogram(OzESI_time_df):
    # Create a copy of the DataFrame to avoid modifying the original DataFrame
    OzESI_rt_df = OzESI_time_df.copy()

    # Round retention time to 1 decimal place
    OzESI_rt_df['Retention_Time'] = round(OzESI_rt_df['Retention_Time'], 2)

    # Create a column called Transition with the Parent_Ion and Product_Ion
    OzESI_rt_df['Transition'] = OzESI_rt_df['Parent_Ion'].astype(str) + ' -> ' + OzESI_rt_df['Product_Ion'].astype(str)

    # Drop retention time values below 10.0 seconds and above 15.5 seconds
    OzESI_rt_df = OzESI_rt_df[(OzESI_rt_df['Retention_Time'] > 10.0) & (OzESI_rt_df['Retention_Time'] < 15.5)]

    # Get the sum of 'OzESI_Intensity' for each 'Transition'
    OzESI_rt_df_top = OzESI_rt_df.groupby('Transition').agg({
        'Parent_Ion': 'first',
        'Product_Ion': 'first',
        'Retention_Time': 'first',
        'OzESI_Intensity': 'sum'
    }).reset_index()

    print('Sum of OzESI Intensity per transition:\n', OzESI_rt_df_top)

    return OzESI_rt_df_top

#Taking the sum of intensity for each transition instead of the max value
# def process_chromatogram_sum(OzESI_time):
#     # Create a copy of the DataFrame to avoid modifying the original DataFrame
#     OzESI_rt_df = OzESI_time.copy()

#     # Round retention Retention_Time to 1 decimal place
#     OzESI_rt_df['Retention_Time'] = round(OzESI_rt_df['Retention_Time'], 2)

#     # Create a column called Transition with the Parent_Ion and Product_Ion
#     OzESI_rt_df['Transition'] = OzESI_rt_df['Parent_Ion'].astype(str) + ' -> ' + OzESI_rt_df['Product_Ion'].astype(str)

#     # Drop Retention_Time values below 10.0 seconds and above 15.5 seconds
#     OzESI_rt_df = OzESI_rt_df[(OzESI_rt_df['Retention_Time'] > 10.0) & (OzESI_rt_df['Retention_Time'] < 15.5)]

#     # Calculate the sum of 'OzESI_Intensity' for each 'Transition'
#     OzESI_rt_df = OzESI_rt_df.groupby('Transition').agg({'Parent_Ion': 'first', 'Product_Ion': 'first', 'Retention_Time': 'first', 'OzESI_Intensity': 'sum'}).reset_index()

#     print('Sum of OzESI Intensity per transition:\n', OzESI_rt_df)

#     return OzESI_rt_df


def add_rt_intensity(df, OzESI_rt_df_top):

    #### STRING OF RETENTION TIMES
    # Group by 'Transition' and get lists of retention times and intensities
    transitions_to_rt = OzESI_rt_df_top.groupby('Transition')['Retention_Time'].apply(list).to_dict()
    transitions_to_intensity = OzESI_rt_df_top.groupby('Transition')['Intensity_OzESI'].apply(list).to_dict()
    
    # Use the map function to add retention times and intensities to the dataframe as strings
    df['Retention_Time'] = df['Transition'].map(transitions_to_rt).apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
    df['Intensity_OzESI'] = df['Transition'].map(transitions_to_intensity).apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
    

    #MEAN OF RENETION TIMES
    # Convert the 'Retention_Time' column from string of lists to lists of floats
    df['Retention_Time'] = df['Retention_Time'].apply(lambda x: [float(i) for i in x.split(', ')] if isinstance(x, str) else x)

    # Calculate the mean of 'Retention_Time' for each transition
    df['Mean_Retention_Time'] = df['Retention_Time'].apply(np.mean).round(2)

    # Convert the 'Intensity_OzESI' column from string of lists to lists of floats
    df['Intensity_OzESI'] = df['Intensity_OzESI'].apply(lambda x: [float(i) for i in x.split(', ')] if isinstance(x, str) else x)

    # Calculate the mean of 'Intensity_OzESI' for each transition
    df['Mean_Intensity_OzESI'] = df['Intensity_OzESI'].apply(np.mean)

    return df

def create_aldehyde_ion_dataframe():
    # Create a pandas dataframe with columns for DB_Position and Aldehyde_Ion
    df_OzESI = pd.DataFrame(columns=['DB_Position','Aldehyde_Ion'])

    # Loop over the range of DB_Position values and calculate the corresponding Aldehyde_Ion values
    for i in range(3, 21):
        df_OzESI.loc[i,'DB_Position'] = i
        df_OzESI.loc[i,'Aldehyde_Ion'] = 26 + (14 * (i-3)) 

    # Print the first 25 rows of the dataframe
    # print(df_OzESI.head(25))

    # Return the dataframe
    return df_OzESI

#OzESI_list = [3,5,7,9,11]
OzESI_list = [7,9,12]
#def calculate_n_minus_values(df_matched, df_OzESI, OzESI_list=[3,5,7,9,11], starting_column=9, last_column=14):
def calculate_n_minus_values(df_matched, df_OzESI, OzESI_list=[7, 9, 12], starting_column=9, last_column=14):
    """
    Given a pandas dataframe df_matched and a dataframe df_OzESI containing DB_Position and Aldehyde_Ion values,
    calculates the n-i values for each i in OzESI_list by subtracting the corresponding Aldehyde_Ion value
    from the Parent_Ion column in df_matched and storing the result in new columns named 'n-i' in df_matched.
    """
    
    # Iterate through OzESI_list
    for i in OzESI_list:
        # Retrieve the Aldehyde_Ion value for the current DB_Position
        aldehyde_ion = df_OzESI.loc[df_OzESI["DB_Position"] == i, "Aldehyde_Ion"].values[0]
        
        # Calculate n-i values
        df_matched["n-{}".format(i)] = df_matched["Parent_Ion"] - aldehyde_ion
    
    return df_matched




In [9]:
df_OzESI_processed['Labels'] = ''
#df_OzESI_processed.head(None)

In [10]:
#### SECOND VERSION (working better)
OzESI_list = [7, 9, 12]

def within_tolerance(a, b, tolerance=0.3):
    return abs(a - b) <= tolerance
columns = [
    'Lipid', 'Parent_Ion', 'Product_Ion', 'Intensity', 'Transition', 'Class',
    'Sample_ID', 'Retention_Time', 'Intensity_OzESI', 'Mean_Retention_Time',
    'Mean_Intensity_OzESI', 'n-7', 'n-9', 'n-12', 'Labels'
]
df_OzESI_n = pd.DataFrame(columns=columns)

### FIRST VERSION (not working)
# def add_lipid_info(df, OzESI_list, tolerance=0.3):
#     df_test = df.copy()
    
#     for i in OzESI_list:
#         df_test['n-' + str(i)] = df_test['n-' + str(i)].astype(float)
    
#     for i in range(len(df_test)):
#         if pd.isna(df_test.loc[i, 'Lipid']):
#             parent_ion = df_test.loc[i, 'Parent_Ion']
            
#             for j in range(len(df_test)):
#                 if within_tolerance(parent_ion, df_test.loc[j, 'n-7'], tolerance) and isinstance(df_test.loc[j, 'Lipid'], str):
#                     df_test.loc[i, 'Lipid'] = df_test.loc[j, 'Lipid']
#                     df_test.loc[i, 'Labels'] = 'n-7' + df_test.loc[j, 'Labels']
#                 elif within_tolerance(parent_ion, df_test.loc[j, 'n-9'], tolerance) and isinstance(df_test.loc[j, 'Lipid'], str):
#                     df_test.loc[i, 'Lipid'] = df_test.loc[j, 'Lipid']
#                     df_test.loc[i, 'Labels'] = 'n-9' + df_test.loc[j, 'Labels']
#                 elif within_tolerance(parent_ion, df_test.loc[j, 'n-12'], tolerance) and isinstance(df_test.loc[j, 'Lipid'], str):
#                     df_test.loc[i, 'Lipid'] = df_test.loc[j, 'Lipid']
#                     df_test.loc[i, 'Labels'] = 'n-12' + df_test.loc[j, 'Labels']
    
#     df_test.dropna(subset=['Lipid'], inplace=True)
#     return df_test


# #### SECOND VERSION (working better) appends to df_OzESI_n, but duplicates the Intensity values
# def add_lipid_info(df, OzESI_list, tolerance=0.3):
#     df_test = df.copy()
#     global df_OzESI_n
    
#     for i in OzESI_list:
#         df_test['n-' + str(i)] = df_test['n-' + str(i)].astype(float)
    
#     for i in range(len(df_test)):
#         if pd.isna(df_test.loc[i, 'Lipid']):
#             parent_ion = df_test.loc[i, 'Parent_Ion']
            
#             for j in range(len(df_test)):
#                 row_data = df_test.loc[j].copy()
#                 if within_tolerance(parent_ion, row_data['n-7'], tolerance) and isinstance(row_data['Lipid'], str):
#                     df_test.loc[i, 'Lipid'] = row_data['Lipid']
#                     df_test.loc[i, 'Labels'] = 'n-7' + row_data['Labels']
                    
#                     # Append to df_OzESI_n
#                     row_data['Labels'] = 'n-7' + row_data['Labels']
#                     df_OzESI_n = df_OzESI_n.append(row_data, ignore_index=True)
                    
#                 elif within_tolerance(parent_ion, row_data['n-9'], tolerance) and isinstance(row_data['Lipid'], str):
#                     df_test.loc[i, 'Lipid'] = row_data['Lipid']
#                     df_test.loc[i, 'Labels'] = 'n-9' + row_data['Labels']
                    
#                     # Append to df_OzESI_n
#                     row_data['Labels'] = 'n-9' + row_data['Labels']
#                     df_OzESI_n = df_OzESI_n.append(row_data, ignore_index=True)
                    
#                 elif within_tolerance(parent_ion, row_data['n-12'], tolerance) and isinstance(row_data['Lipid'], str):
#                     df_test.loc[i, 'Lipid'] = row_data['Lipid']
#                     df_test.loc[i, 'Labels'] = 'n-12' + row_data['Labels']
                    
#                     # Append to df_OzESI_n
#                     row_data['Labels'] = 'n-12' + row_data['Labels']
#                     df_OzESI_n = df_OzESI_n.append(row_data, ignore_index=True)
    
#     df_test.dropna(subset=['Lipid'], inplace=True)
#     return df_test

# # Call the function (assuming you have defined the function within_tolerance before)
# # updated_df = add_lipid_info(original_df, OzESI_list)

def add_lipid_info(df, OzESI_list, tolerance=0.3):
    df_test = df.copy()
    global df_OzESI_n
    
    for i in OzESI_list:
        df_test['n-' + str(i)] = df_test['n-' + str(i)].astype(float)
    
    for i in range(len(df_test)):
        if pd.isna(df_test.loc[i, 'Lipid']):
            parent_ion = df_test.loc[i, 'Parent_Ion']
            
            for j in range(len(df_test)):
                row_data = df_test.loc[j].copy()
                if within_tolerance(parent_ion, row_data['n-7'], tolerance) and isinstance(row_data['Lipid'], str):
                    df_test.loc[i, 'Lipid'] = row_data['Lipid']
                    df_test.loc[i, 'Labels'] = 'n-7' + row_data['Labels']
                    
                    # Append to df_OzESI_n
                    appended_row = df_test.loc[i].copy()
                    appended_row['Labels'] = 'n-7' + row_data['Labels']
                    df_OzESI_n = df_OzESI_n.append(appended_row, ignore_index=True)
                    
                elif within_tolerance(parent_ion, row_data['n-9'], tolerance) and isinstance(row_data['Lipid'], str):
                    df_test.loc[i, 'Lipid'] = row_data['Lipid']
                    df_test.loc[i, 'Labels'] = 'n-9' + row_data['Labels']
                    
                    # Append to df_OzESI_n
                    appended_row = df_test.loc[i].copy()
                    appended_row['Labels'] = 'n-9' + row_data['Labels']
                    df_OzESI_n = df_OzESI_n.append(appended_row, ignore_index=True)
                    
                elif within_tolerance(parent_ion, row_data['n-12'], tolerance) and isinstance(row_data['Lipid'], str):
                    df_test.loc[i, 'Lipid'] = row_data['Lipid']
                    df_test.loc[i, 'Labels'] = 'n-12' + row_data['Labels']
                    
                    # Append to df_OzESI_n
                    appended_row = df_test.loc[i].copy()
                    appended_row['Labels'] = 'n-12' + row_data['Labels']
                    df_OzESI_n = df_OzESI_n.append(appended_row, ignore_index=True)
    
    df_test.dropna(subset=['Lipid'], inplace=True)
    return df_test


In [11]:
df_test = add_lipid_info(df_OzESI_processed, OzESI_list)
df_test.head(None)

/tmp/ipykernel_5996/1382359555.py:121: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_OzESI_n = df_OzESI_n.append(appended_row, ignore_index=True)
/tmp/ipykernel_5996/1382359555.py:112: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_OzESI_n = df_OzESI_n.append(appended_row, ignore_index=True)
/tmp/ipykernel_5996/1382359555.py:103: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_OzESI_n = df_OzESI_n.append(appended_row, ignore_index=True)


,Lipid,Parent_Ion,Product_Ion,Intensity,Transition,Class,Sample_ID,Retention_Time,Intensity_OzESI,Mean_Retention_Time,Mean_Intensity_OzESI,n-7,n-9,n-12,Labels
0,[TG(50:3)]_FA18:1,694.6,547.6,2.567996e+05,694.6 -> 547.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,[11.69],[3903.0],11.69,3903.0,612.6,584.6,542.6,n-12
1,"[TG(51:9),TG(50:2)]_FA18:1",696.6,549.6,2.185144e+05,696.6 -> 549.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,[12.91],[18586.0],12.91,18586.0,614.6,586.6,544.6,n-12
2,"[TG(51:8),TG(50:1)]_FA18:1",698.7,551.6,6.608002e+06,698.7 -> 551.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,[14.0],[59211.0],14.00,59211.0,616.7,588.7,546.7,n-12
3,[TG(52:4)]_FA18:1,720.6,573.6,7.330655e+04,720.6 -> 573.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,[13.63],[590.0],13.63,590.0,638.6,610.6,568.6,n-12
4,"[TG(53:10),TG(52:3)]_FA18:1",722.6,575.6,1.428391e+05,722.6 -> 575.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,[12.77],[3643.0],12.77,3643.0,640.6,612.6,570.6,n-12
5,"[TG(53:9),TG(52:2)]_FA18:1",724.7,577.6,5.815955e+05,724.7 -> 577.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,[13.99],[15596.0],13.99,15596.0,642.7,614.7,572.7,n-12
6,[TG(50:3)]_FA18:1,736.7,547.6,6.613184e+05,736.7 -> 547.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,[12.52],[9713.0],12.52,9713.0,654.7,626.7,584.7,n-9
7,"[TG(51:9),TG(50:2)]_FA18:1",738.7,549.6,4.582103e+05,738.7 -> 549.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,[13.74],[49935.0],13.74,49935.0,656.7,628.7,586.7,n-9
8,"[TG(51:8),TG(50:1)]_FA18:1",740.8,551.6,9.366814e+05,740.8 -> 551.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,[14.95],[144069.0],14.95,144069.0,658.8,630.8,588.8,n-9
9,[TG(54:5)]_FA18:1,746.8,599.6,1.976136e+05,746.8 -> 599.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,[15.19],[584.0],15.19,584.0,664.8,636.8,594.8,n-12


In [13]:
df_OzESI_n.head(None)

,Lipid,Parent_Ion,Product_Ion,Intensity,Transition,Class,Sample_ID,Retention_Time,Intensity_OzESI,Mean_Retention_Time,Mean_Intensity_OzESI,n-7,n-9,n-12,Labels
0,[TG(50:3)]_FA18:1,694.6,547.6,2.567996e+05,694.6 -> 547.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,[11.69],[3903.0],11.69,3903.0,612.6,584.6,542.6,n-12
1,"[TG(51:9),TG(50:2)]_FA18:1",696.6,549.6,2.185144e+05,696.6 -> 549.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,[12.91],[18586.0],12.91,18586.0,614.6,586.6,544.6,n-12
2,"[TG(51:8),TG(50:1)]_FA18:1",698.7,551.6,6.608002e+06,698.7 -> 551.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,[14.0],[59211.0],14.00,59211.0,616.7,588.7,546.7,n-12
3,[TG(52:4)]_FA18:1,720.6,573.6,7.330655e+04,720.6 -> 573.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,[13.63],[590.0],13.63,590.0,638.6,610.6,568.6,n-12
4,"[TG(53:10),TG(52:3)]_FA18:1",722.6,575.6,1.428391e+05,722.6 -> 575.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,[12.77],[3643.0],12.77,3643.0,640.6,612.6,570.6,n-12
5,"[TG(53:9),TG(52:2)]_FA18:1",724.7,577.6,5.815955e+05,724.7 -> 577.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,[13.99],[15596.0],13.99,15596.0,642.7,614.7,572.7,n-12
6,[TG(50:3)]_FA18:1,736.7,547.6,6.613184e+05,736.7 -> 547.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,[12.52],[9713.0],12.52,9713.0,654.7,626.7,584.7,n-9
7,"[TG(51:9),TG(50:2)]_FA18:1",738.7,549.6,4.582103e+05,738.7 -> 549.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,[13.74],[49935.0],13.74,49935.0,656.7,628.7,586.7,n-9
8,"[TG(51:8),TG(50:1)]_FA18:1",740.8,551.6,9.366814e+05,740.8 -> 551.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,[14.95],[144069.0],14.95,144069.0,658.8,630.8,588.8,n-9
9,[TG(54:5)]_FA18:1,746.8,599.6,1.976136e+05,746.8 -> 599.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,[15.19],[584.0],15.19,584.0,664.8,636.8,594.8,n-12


In [14]:
# sorted_df = df_test.sort_values(by=['Sample_ID','Product_Ion'])

# sorted_df.head(None)

sorted_df = df_OzESI_n.sort_values(by=['Sample_ID','Product_Ion'])
#drop duplicates Lipid and Labels
sorted_df = sorted_df.drop_duplicates(subset=['Lipid','Labels'], keep='first')
sorted_df.head(None)


,Lipid,Parent_Ion,Product_Ion,Intensity,Transition,Class,Sample_ID,Retention_Time,Intensity_OzESI,Mean_Retention_Time,Mean_Intensity_OzESI,n-7,n-9,n-12,Labels
0,[TG(50:3)]_FA18:1,694.6,547.6,2.567996e+05,694.6 -> 547.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,[11.69],[3903.0],11.69,3903.0,612.6,584.6,542.6,n-12
6,[TG(50:3)]_FA18:1,736.7,547.6,6.613184e+05,736.7 -> 547.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,[12.52],[9713.0],12.52,9713.0,654.7,626.7,584.7,n-9
13,[TG(50:3)]_FA18:1,764.7,547.6,1.187571e+06,764.7 -> 547.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,NaN,NaN,NaN,NaN,682.7,654.7,612.7,n-7
14,"[TG(53:10),TG(52:3)]_FA18:1",764.7,547.6,1.187571e+06,764.7 -> 547.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,NaN,NaN,NaN,NaN,682.7,654.7,612.7,n-9
1,"[TG(51:9),TG(50:2)]_FA18:1",696.6,549.6,2.185144e+05,696.6 -> 549.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,[12.91],[18586.0],12.91,18586.0,614.6,586.6,544.6,n-12
7,"[TG(51:9),TG(50:2)]_FA18:1",738.7,549.6,4.582103e+05,738.7 -> 549.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,[13.74],[49935.0],13.74,49935.0,656.7,628.7,586.7,n-9
17,"[TG(51:9),TG(50:2)]_FA18:1",766.7,549.6,3.307983e+05,766.7 -> 549.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,NaN,NaN,NaN,NaN,684.7,656.7,614.7,n-7
18,"[TG(53:9),TG(52:2)]_FA18:1",766.7,549.6,3.307983e+05,766.7 -> 549.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,NaN,NaN,NaN,NaN,684.7,656.7,614.7,n-9
2,"[TG(51:8),TG(50:1)]_FA18:1",698.7,551.6,6.608002e+06,698.7 -> 551.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,[14.0],[59211.0],14.00,59211.0,616.7,588.7,546.7,n-12
8,"[TG(51:8),TG(50:1)]_FA18:1",740.8,551.6,9.366814e+05,740.8 -> 551.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,[14.95],[144069.0],14.95,144069.0,658.8,630.8,588.8,n-9


In [15]:
# def calculate_intensity_ratio(sorted_df):
#     # Create a new column combining 'Lipid' and 'Labels'
#     sorted_df['Lipid_Labels'] = sorted_df['Lipid'] + '_' + sorted_df['Labels']

#     # Create an empty dictionary to hold the ratios
#     ratio_dic = {}

#     # Get the unique lipid names
#     unique_lipids = sorted_df['Lipid'].unique()

#     # Loop through each unique lipid
#     for lipid in unique_lipids:
#         # Get the rows with the lipid name and 'n-7' label
#         n7_rows = sorted_df[(sorted_df['Lipid'] == lipid) & (sorted_df['Labels'].str.contains('n-7'))]
#         # Get the rows with the lipid name and 'n-9' label
#         n9_rows = sorted_df[(sorted_df['Lipid'] == lipid) & (sorted_df['Labels'].str.contains('n-9'))]

#         # If there is at least one row with 'n-7' label and one row with 'n-9' label for the lipid
#         if len(n7_rows) > 0 and len(n9_rows) > 0:
#             # Calculate the mean intensity for 'n-7' and 'n-9'
#             n7_intensity = n7_rows['Intensity'].mean()
#             n9_intensity = n9_rows['Intensity'].mean()

#             # If the mean intensity for 'n-9' is not zero
#             if n9_intensity != 0:
#                 # Calculate the ratio and store it in the dictionary
#                 ratio_dic[lipid] = n9_intensity / n7_intensity

#     # Remove the 'Lipid_Labels' column
#     sorted_df.drop(columns='Lipid_Labels', inplace=True)

#     # Return the dictionary
#     return ratio_dic

# ratio_dic = calculate_intensity_ratio(sorted_df)
# for lipid, ratio in ratio_dic.items():
#     print(f'Lipid: {lipid}, Intensity Ratio (n-9/n-7): {ratio.round(2)}')

# def compute_intensity_ratio(df):
#     df = df.copy()  # make a copy of the dataframe to avoid changing the original one
#     df['Ratio'] = None  # initially set all ratios to None
#     unique_lipids = df['Lipid'].unique()
#     for lipid in unique_lipids:
#         lipid_rows = df[df['Lipid'] == lipid]
#         if 'n-7' in lipid_rows['Labels'].values and 'n-9' in lipid_rows['Labels'].values:
#             intensity_n7 = lipid_rows[lipid_rows['Labels'] == 'n-7']['Intensity'].values[0]
#             intensity_n9 = lipid_rows[lipid_rows['Labels'] == 'n-9']['Intensity'].values[0]
#             if intensity_n9 != 0:  # to avoid division by zero
#                 intensity_ratio = round(intensity_n7 / intensity_n9, 2)  # rounding to 2 decimal places
#                 df.loc[lipid_rows[lipid_rows['Labels'] == 'n-7'].index, 'Ratio'] = intensity_ratio
#     return df

# sorted_df = compute_intensity_ratio(sorted_df)
# sorted_df.head(None)


# #First Version WOrking kinda, but dividng by intensity instead of OzESI intensity
# def compute_intensity_ratio(df):
#     df = df.copy()  # make a copy of the dataframe to avoid changing the original one
#     df['Ratio'] = None  # initially set all ratios to None
#     unique_lipids = df['Lipid'].unique()
#     for lipid in unique_lipids:
#         lipid_rows = df[df['Lipid'] == lipid]
#         if 'n-7' in lipid_rows['Labels'].values and 'n-9' in lipid_rows['Labels'].values:
#             # check if the 'Sample_ID' values are the same for 'n-7' and 'n-9' lipid rows
#             sample_id_n7 = lipid_rows[lipid_rows['Labels'] == 'n-7']['Sample_ID'].values[0]
#             sample_id_n9 = lipid_rows[lipid_rows['Labels'] == 'n-9']['Sample_ID'].values[0]
#             if sample_id_n7 == sample_id_n9:
#                 intensity_n7 = lipid_rows[lipid_rows['Labels'] == 'n-7']['Intensity'].values[0]
#                 intensity_n9 = lipid_rows[lipid_rows['Labels'] == 'n-9']['Intensity'].values[0]
#                 if intensity_n9 != 0:  # to avoid division by zero
#                     intensity_ratio = round(intensity_n9 / intensity_n7, 2)  # rounding to 2 decimal places
#                     df.loc[lipid_rows[lipid_rows['Labels'] == 'n-7'].index, 'Ratio'] = intensity_ratio
#                     print(f'Lipid: {lipid}, Intensity Ratio (n-9/n-7): {intensity_ratio}')
#     return df

# sorted_df = compute_intensity_ratio(sorted_df)
# sorted_df.head(25)

def compute_intensity_ratio(df):
    df = df.copy()  # make a copy of the dataframe to avoid changing the original one
    df['Ratio'] = None  # initially set all ratios to None
    unique_lipids = df['Lipid'].unique()
    for lipid in unique_lipids:
        lipid_rows = df[df['Lipid'] == lipid]
        if 'n-7' in lipid_rows['Labels'].values and 'n-9' in lipid_rows['Labels'].values:
            # check if the 'Sample_ID' values are the same for 'n-7' and 'n-9' lipid rows
            sample_id_n7 = lipid_rows[lipid_rows['Labels'] == 'n-7']['Sample_ID'].values[0]
            sample_id_n9 = lipid_rows[lipid_rows['Labels'] == 'n-9']['Sample_ID'].values[0]
            if sample_id_n7 == sample_id_n9:
                intensity_n7 = lipid_rows[lipid_rows['Labels'] == 'n-7']['Intensity_OzESI'].values[0]
                intensity_n9 = lipid_rows[lipid_rows['Labels'] == 'n-9']['Intensity_OzESI'].values[0]
                
                # Extract scalar values from lists if they are in list format
                if isinstance(intensity_n7, list) and len(intensity_n7) > 0:
                    intensity_n7 = intensity_n7[0]
                if isinstance(intensity_n9, list) and len(intensity_n9) > 0:
                    intensity_n9 = intensity_n9[0]

                # Check if the intensities are available and non-zero before dividing
                if intensity_n7 and intensity_n9 and intensity_n7 != 0:
                    intensity_ratio = round(intensity_n9 / intensity_n7, 2)  # rounding to 2 decimal places
                    df.loc[lipid_rows[lipid_rows['Labels'] == 'n-7'].index, 'Ratio'] = intensity_ratio
                    print(f'Lipid: {lipid}, Intensity Ratio (n-9/n-7): {intensity_ratio}')
    return df

sorted_df = compute_intensity_ratio(sorted_df)
sorted_df.head(25)


Lipid: [TG(50:3)]_FA18:1, Intensity Ratio (n-9/n-7): nan
Lipid: [TG(53:10),TG(52:3)]_FA18:1, Intensity Ratio (n-9/n-7): nan
Lipid: [TG(51:9),TG(50:2)]_FA18:1, Intensity Ratio (n-9/n-7): nan
Lipid: [TG(53:9),TG(52:2)]_FA18:1, Intensity Ratio (n-9/n-7): nan
Lipid: [TG(51:8),TG(50:1)]_FA18:1, Intensity Ratio (n-9/n-7): 1.73
Lipid: [TG(52:4)]_FA18:1, Intensity Ratio (n-9/n-7): 3.05
Lipid: [TG(55:11),TG(54:4)]_FA18:1, Intensity Ratio (n-9/n-7): 0.42
Lipid: [TG(55:10),TG(54:3)]_FA18:1, Intensity Ratio (n-9/n-7): 0.72
Lipid: [TG(54:5)]_FA18:1, Intensity Ratio (n-9/n-7): 3.0


,Lipid,Parent_Ion,Product_Ion,Intensity,Transition,Class,Sample_ID,Retention_Time,Intensity_OzESI,Mean_Retention_Time,Mean_Intensity_OzESI,n-7,n-9,n-12,Labels,Ratio
0,[TG(50:3)]_FA18:1,694.6,547.6,2.567996e+05,694.6 -> 547.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,[11.69],[3903.0],11.69,3903.0,612.6,584.6,542.6,n-12,None
6,[TG(50:3)]_FA18:1,736.7,547.6,6.613184e+05,736.7 -> 547.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,[12.52],[9713.0],12.52,9713.0,654.7,626.7,584.7,n-9,None
13,[TG(50:3)]_FA18:1,764.7,547.6,1.187571e+06,764.7 -> 547.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,NaN,NaN,NaN,NaN,682.7,654.7,612.7,n-7,NaN
14,"[TG(53:10),TG(52:3)]_FA18:1",764.7,547.6,1.187571e+06,764.7 -> 547.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,NaN,NaN,NaN,NaN,682.7,654.7,612.7,n-9,None
1,"[TG(51:9),TG(50:2)]_FA18:1",696.6,549.6,2.185144e+05,696.6 -> 549.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,[12.91],[18586.0],12.91,18586.0,614.6,586.6,544.6,n-12,None
7,"[TG(51:9),TG(50:2)]_FA18:1",738.7,549.6,4.582103e+05,738.7 -> 549.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,[13.74],[49935.0],13.74,49935.0,656.7,628.7,586.7,n-9,None
17,"[TG(51:9),TG(50:2)]_FA18:1",766.7,549.6,3.307983e+05,766.7 -> 549.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,NaN,NaN,NaN,NaN,684.7,656.7,614.7,n-7,NaN
18,"[TG(53:9),TG(52:2)]_FA18:1",766.7,549.6,3.307983e+05,766.7 -> 549.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,NaN,NaN,NaN,NaN,684.7,656.7,614.7,n-9,None
2,"[TG(51:8),TG(50:1)]_FA18:1",698.7,551.6,6.608002e+06,698.7 -> 551.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,[14.0],[59211.0],14.00,59211.0,616.7,588.7,546.7,n-12,None
8,"[TG(51:8),TG(50:1)]_FA18:1",740.8,551.6,9.366814e+05,740.8 -> 551.6,NaN,DOD93_F4-5xFAD-Cereb_TG18-1_o3on,[14.95],[144069.0],14.95,144069.0,658.8,630.8,588.8,n-9,None


In [16]:
sorted_df.to_excel('data_results/data/data_matching/TEST_04-29-2023_validation_test_2/TEST_04-29-2023_validation_test_4.xlsx', index=False)

In [ ]:
import plotly.express as px

# Remove rows with NaN values in the 'Ratio' column
filtered_df = sorted_df.dropna(subset=['Ratio'])

fig = px.bar(filtered_df, x='Lipid', y='Ratio', color='Sample_ID', title='Ratio vs Lipid', barmode='group')
fig.show()



# fig = px.bar(sorted_df, x='Lipid', y='Ratio', color='Sample_ID', title='Ratio vs Lipid', barmode='group')
# fig.show()

# import plotly.figure_factory as ff

# # Create pivot table
# pivot_df = sorted_df.pivot_table(values='Ratio', index='Lipid', columns='Sample_ID')

# # Plot heatmap
# fig = ff.create_annotated_heatmap(z=pivot_df.values, x=list(pivot_df.columns), y=pivot_df.index.tolist(), colorscale='Viridis')
# fig.show()
# import plotly.graph_objects as go

# # Create pivot table
# pivot_df = sorted_df.pivot_table(values='Ratio', index='Lipid', columns='Sample_ID')

# # Plot heatmap
# fig = go.Figure(data=go.Heatmap(
#                    z=pivot_df.values,
#                    x=pivot_df.columns.tolist(),
#                    y=pivot_df.index.tolist(),
#                    colorscale='Viridis',
#                    colorbar=dict(title="Ratio")))

# fig.update_layout(title='Ratio Heatmap', xaxis_nticks=len(pivot_df.columns))
# fig.show()


In [ ]:
print(ratio_dic)
# sorted_df.head(None)
print(sorted_df[sorted_df['Labels'].str.contains('n-7')].head())
print(sorted_df[sorted_df['Labels'].str.contains('n-9')].head())


In [ ]:
print(sorted_df[['Lipid', 'Mean_Retention_Time','Retention_Time']])


In [ ]:
sorted_df[['Lipid', 'Retention_Time']].head(None)

In [ ]:
# for index, row in sorted_df.iterrows():
#     print(f"Lipid: {row['Lipid']}, Mean Retention Time: {row['Mean_Retention_Time']}")

for index, row in sorted_df.iterrows():
    print(f"Lipid: {row['Lipid']}, Mean Retention Time: {row['Mean_Retention_Time']}")
    print(f"Retention Time: {row['Retention_Time']}")
    print('________________________________________________________________________________________')

In [ ]:
data = {
    "Lipid": ["TG 50:1", "TG 50:1", "TG 50:2", "TG 50:2", "TG 52:2", "TG 52:2", "TG 52:3", "TG 52:3", "TG 52:3", "TG 52:4", "TG 52:4", "TG 54:3", "TG 54:3", "TG 54:4", "TG 54:4", "TG 54:5", "TG 54:5"],
    "MRM Identity": ["TG 50:1_FA 18:1n-9", "TG 50:1_FA 18:1n-7", "TG 50:2_FA 18:1n-9", "TG 50:2_FA 18:1n-7", "TG 52:2_FA 18:1n-9", "TG 52:2_FA 18:1n-7", "TG 52:3_FA 18:1n-12", "TG 52:3_FA 18:1n-9", "TG 52:3_FA 18:1n-7", "TG 52:4_FA 18:1n-9", "TG 52:4_FA 18:1n-7", "TG 54:3_FA 18:1n-9", "TG 54:3_FA 18:1n-7", "TG 54:4_FA 18:1n-9", "TG 54:4_FA 18:1n-7", "TG 54:5_FA 18:1n-7", "TG 54:5_FA 18:1n-9"],
    "MRM Transition": ["740.8 -> 551.6", "768.8 -> 551.6", "740.8 -> 549.6", "768.8 -> 549.6", "766.8 -> 577.6", "794.8 -> 577.6", "722.7 -> 575.6", "764.8 -> 575.6", "792.8 -> 575.6", "762.8 -> 573.6", "790.8 -> 573.6", "792.8 -> 603.6", "820.8 -> 603.6", "790.8 -> 601.6", "818.8 -> 601.6", "816.8 -> 599.6", "788.8 -> 599.6"],
    "Retention Time": [15.1, 15.1, 13.8, 13.8, 14.9, 15.0, np.nan, 13.7, 13.7, 12.5, 12.5, 14.9, 14.8, 13.6, 13.7, 12.6, 12.6]
}

C_df = pd.DataFrame(data)

C_df.head(None)


In [ ]:
# import plotly.graph_objects as go

# fig = go.Figure()

# # For each unique transition in C_df, create a separate trace
# for transition in C_df['MRM Transition'].unique():
#     df = C_df[C_df['MRM Transition'] == transition]
#     fig.add_trace(go.Scatter(
#         y=df['Retention Time'], 
#         x=df['MRM Transition'],
#         mode='markers',
#         name=transition
#     ))

# # For each unique transition in sorted_df, create a separate trace
# for transition in sorted_df['Transition'].unique():
#     df = sorted_df[sorted_df['Transition'] == transition]
#     fig.add_trace(go.Scatter(
#         y=df['Mean_Retention_Time'], 
#         x=df['Transition'],
#         mode='markers',
#         name=transition
#     ))

# # Define the layout
# fig.update_layout(
#     title='Lipid vs. Retention Time',
#     yaxis_title='Retention Time',
#     xaxis_title='Lipid'
# )

# # Show the plot
# fig.show()

# import plotly.graph_objects as go

# fig = go.Figure()

# # For each unique transition in C_df, create a separate trace
# for transition in C_df['MRM Transition'].unique():
#     df = C_df[C_df['MRM Transition'] == transition]
#     fig.add_trace(go.Scatter(
#         y=df['Retention Time'], 
#         x=df['MRM Transition'],
#         mode='markers',
#         name=transition,
#         marker=dict(color='red')  # Set color to red for C_df
#     ))

# # For each unique transition in sorted_df, create a separate trace
# for transition in sorted_df['Transition'].unique():
#     df = sorted_df[sorted_df['Transition'] == transition]
#     fig.add_trace(go.Scatter(
#         y=df['Mean_Retention_Time'], 
#         x=df['Transition'],
#         mode='markers',
#         name=transition,
#         marker=dict(color='blue')  # Set color to blue for sorted_df
#     ))

# # Define the layout
# fig.update_layout(
#     title='Lipid vs. Retention Time',
#     yaxis_title='Retention Time',
#     xaxis_title='Lipid'
# )

# # Show the plot
# fig.show()

import plotly.graph_objects as go

fig = go.Figure()

# For each unique transition in C_df, create a separate trace
for transition in C_df['MRM Transition'].unique():
    df = C_df[C_df['MRM Transition'] == transition]
    fig.add_trace(go.Scatter(
        y=df['Retention Time'], 
        x=df['MRM Transition'],
        mode='markers',
        name=transition,
        hovertext=df['Lipid'],
        marker=dict(color='red')  # Set color to red for C_df
    ))

# For each unique transition in sorted_df, create a separate trace
for transition in sorted_df['Transition'].unique():
    df = sorted_df[sorted_df['Transition'] == transition]
    fig.add_trace(go.Scatter(
        y=df['Mean_Retention_Time'], 
        x=df['Transition'],
        mode='markers',
        name=transition,
        hovertext=df['Lipid'],
        marker=dict(color='blue')  # Set color to blue for sorted_df
    ))

# Define the layout
fig.update_layout(
    title='Lipid vs. Retention Time',
    yaxis_title='Retention Time',
    xaxis_title='Lipid'
)

# Show the plot
fig.show()
